<a href="https://colab.research.google.com/github/zahraniayudyaa/midterm-machine-learning/blob/main/Zahrani_UTS_ML_NoTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install xgboost==2.0.3 polars
!pip install optuna
!pip install polars

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.1/297.1 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 783.6/783.6 kB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.7/404.7 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.6/248.6 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 607.6/607.6 kB 48.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import polars as pl

In [ ]:
!pip install -q gdown

# kalian download semua file dari gdrive dibawah lalu buat folder (ex. midterm_folder)
!gdown --folder 1qoyfQ0yXp9pgnp9zKwsRm0B-u0n-KopN -O midterm_folder

BASE_PATH = "midterm_folder"

train_transaction = pl.read_csv(f"{BASE_PATH}/train_transaction.csv")
test_transaction  = pl.read_csv(f"{BASE_PATH}/test_transaction.csv")

# print(train_transaction.shape) #590540 row dan 393 feature + 1 target
# print(test_transaction.shape) #506691 row dan 393 feature

train_transaction.head() #isFraud adalah target featurenya
test_transaction.head() #uji data baru, setelah pembuatan model machine learningnya

Retrieving folder contents
Processing file 1FlTQDZijxi_AdhdPlxvcgKdSRlcbVrKk test_transaction.csv
Processing file 18BxWO8J8QgLfWeFInaLnswSph4yNg2RL train_transaction.csv
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1FlTQDZijxi_AdhdPlxvcgKdSRlcbVrKk
From (redirected): https://drive.google.com/uc?id=1FlTQDZijxi_AdhdPlxvcgKdSRlcbVrKk&confirm=t&uuid=cac1385d-30fb-43e0-9411-5ae301c3d3ae
To: /content/midterm_folder/test_transaction.csv
100% 613M/613M [00:04<00:00, 126MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=18BxWO8J8QgLfWeFInaLnswSph4yNg2RL
From (redirected): https://drive.google.com/uc?id=18BxWO8J8QgLfWeFInaLnswSph4yNg2RL&confirm=t&uuid=c090b93c-8075-451e-9fc2-0ff84213dc89
To: /content/midterm_folder/train_transaction.csv
100% 683M/683M [00:05<00:00, 131MB/s]
Download completed


TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,…,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
i64,i64,f64,str,i64,f64,f64,str,f64,str,f64,f64,f64,f64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
3663549,18403224,31.95,"""W""",10409,111.0,150.0,"""visa""",226.0,"""debit""",170.0,87.0,1.0,null,"""gmail.com""",null,6.0,6.0,0.0,0.0,3.0,4.0,0.0,0.0,6.0,0.0,5.0,1.0,115.0,6.0,419.0,419.0,27.0,398.0,27.0,null,null,…,0.0,0.0,1.0,0.0,47.950001,0.0,0.0,47.950001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3663550,18403263,49.0,"""W""",4272,111.0,150.0,"""visa""",226.0,"""debit""",299.0,87.0,4.0,null,"""aol.com""",null,3.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,1.0,12.0,2.0,149.0,149.0,7.0,634.0,7.0,null,null,…,0.0,0.0,1.0,0.0,280.0,77.0,0.0,280.0,0.0,77.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3663551,18403310,171.0,"""W""",4476,574.0,150.0,"""visa""",226.0,"""debit""",472.0,87.0,2635.0,null,"""hotmail.com""",null,2.0,2.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,0.0,2.0,0.0,22.0,2.0,137.0,137.0,10.0,97.0,10.0,null,null,…,0.0,0.0,1.0,0.0,1321.0,0.0,0.0,1058.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,263.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3663552,18403310,284.95,"""W""",10989,360.0,150.0,"""visa""",166.0,"""debit""",205.0,87.0,17.0,null,"""gmail.com""",null,5.0,2.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,2.0,0.0,7.0,4.0,42.0,42.0,41.0,242.0,41.0,null,null,…,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,282.540009,282.540009,282.540009,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
3663553,18403317,67.95,"""W""",18018,452.0,150.0,"""mastercard""",117.0,"""debit""",264.0,87.0,6.0,null,"""gmail.com""",null,6.0,6.0,0.0,0.0,2.0,5.0,0.0,0.0,5.0,0.0,6.0,0.0,14.0,6.0,22.0,22.0,0.0,22.0,0.0,null,null,…,0.0,0.0,1.0,67.949997,183.850006,67.949997,67.949997,183.850006,67.949997,67.949997,67.949997,183.850006,67.949997,0.0,0.0,0.0,0.0,0.0,0.0,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null


In [ ]:
!pip install optuna-integration[lightgbm]
!pip install --upgrade lightgbm optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 10.0 MB/s eta 0:00:00


In [ ]:
# ======================================================
# 1. IMPORT LIBRARIES
# ======================================================
import polars as pl
import lightgbm as lgb
from lightgbm import LGBMClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score
import optuna
import os
import optuna.integration.lightgbm as lgb_tuner

# from optuna.integration import lightgbm as lgb_tuner
# from lightgbm import LightGBMTunerCV
from sklearn.model_selection import StratifiedKFold

In [ ]:
# ======================================================
# 2. LOAD DATASET WITH POLARS
# ======================================================
# train_transactions = pl.read_csv("train_transaction.csv")
# test_transactions  = pl.read_csv("test_transaction.csv")

target = "isFraud"

In [ ]:
# =======================================================
# 3. SPLIT NUMERIC & CATEGORICAL COLUMNS & IMPUTATION
# =======================================================
num_cols = [c for c, t in zip(train_transaction.columns, train_transaction.dtypes)
            if t in (pl.Float32, pl.Float64, pl.Int64, pl.Int32) and c != target]

cat_cols = [c for c, t in zip(train_transaction.columns, train_transaction.dtypes)
            if t == pl.Utf8]

# numeric → median
for col in num_cols:
    med = train_transaction[col].median()
    train_transaction = train_transaction.with_columns(pl.col(col).fill_null(med))
    test_transaction  = test_transaction.with_columns(pl.col(col).fill_null(med))

# categorical → missing
for col in cat_cols:
    train_transaction = train_transaction.with_columns(pl.col(col).fill_null("missing"))
    test_transaction  = test_transaction.with_columns(pl.col(col).fill_null("missing"))

In [ ]:
# =======================================================
# 4. LABEL ENCODING (HANYA POLARS)
# =======================================================
for col in cat_cols:
    # Ambil seluruh nilai unik
    unique_vals = (
        pl.concat([train_transaction[col], test_transaction[col]])
        .unique()
        .to_list()
    )

    # Buat mapping integer
    mapping = {v: i for i, v in enumerate(unique_vals)}

    # Terapkan ke train & test (Polars)
    train_transaction = train_transaction.with_columns(
        pl.col(col).replace(mapping).cast(pl.Int64)
    )
    test_transaction = test_transaction.with_columns(
        pl.col(col).replace(mapping).cast(pl.Int64)
    )

In [ ]:
# ======================================================
# 5. SAVE CLEANED DATA TO CSV (FOR STREAMING MODE)
# ======================================================

train_transaction.write_csv("train_clean.csv")
test_transaction.write_csv("test_clean.csv")

# Save labels separately (LightGBM needs it)
y_list = train_transaction[target].to_list()
pl.DataFrame({"isFraud": y_list}).write_csv("train_label.csv")

feature_cols = [c for c in train_transaction.columns if c != target]


train_transaction = train_transaction.sample(fraction=1.0, shuffle=True)

val_size = int(0.2 * train_transaction.height)

val_df = train_transaction[:val_size]
train_transaction2 = train_transaction[val_size:]

X_train = train_transaction2.drop(target).to_numpy()
y_train = train_transaction2[target].to_numpy()

X_val   = val_df.drop(target).to_numpy()
y_val   = val_df[target].to_numpy()


train_lgb = lgb.Dataset(X_train, label=y_train)
valid_lgb = lgb.Dataset(X_val, label=y_val)

In [ ]:
# ======================================================
# 6. LIGHTGBM STREAMING DATASET
# ======================================================

train_data = lgb.Dataset(
    "train_clean.csv",
    label="train_label.csv",
    params={
        "header": True,
        "label_column": "isFraud",
    }
)

In [ ]:
# =======================================================
# 5. CONVERT TO LIGHTGBM DATASET ONCE (ANTI-CRASH)
# =======================================================
X = train_transaction.drop(target).to_pandas()  # safe (done once)
y = train_transaction[target].to_pandas()

train_lgb = lgb.Dataset(X, label=y)

In [ ]:
# ======================================================
# 8. TRAIN FINAL MODEL (STREAMING MODE)
# ======================================================
params = {
    "objective": "binary",
    "metric": "auc",
    "verbosity": -1,
    "boosting_type": "gbdt",
}

final_model = lgb.train(params, train_lgb)
print(f"Type of final_model after training: {type(final_model)}")

Type of final_model after training: <class 'lightgbm.basic.Booster'>


In [ ]:
test_pred = final_model.predict(test_transaction)

submission = pl.DataFrame({
    "TransactionID": test_transaction["TransactionID"],
    "isFraud": test_pred
})

submission.write_csv("submission_done.csv")
print("Done! Saved as submission_done.csv")

/usr/local/lib/python3.12/dist-packages/lightgbm/basic.py:1238: UserWarning: Converting data to scipy sparse matrix.
  _log_warning("Converting data to scipy sparse matrix.")


Done! Saved as submission_done.csv
